In [ ]:
!pip3 install -q -U openpyxl

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
import random
import copy
import typing
from datetime import datetime
from collections import Counter, defaultdict
import openpyxl

# Visualization packages
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# Append system path
sys.path = [p for p in sys.path if not p.endswith('../..')]  # Cleans duplicated '../..'
sys.path.insert(0, '../')  # This adds `src` to the path

from helpers import io, filters, constants
from analysis import analysis_util, analysis_constants, visualization_util
from web_analysis import parse_robots
from web_analysis import robots_util, forecasting_util


%load_ext autoreload
%autoreload 2

### Define Paths to all relevant files

In [ ]:
EXCEL_FNAME_ROBOTS = "data/forecasted_robots_data.xlsx"
EXCEL_FNAME_TOS = "data/forecasted_tos_data.xlsx"
EXCEL_FNAME_FINAL = "data/forecasted_final_data.xlsx"
FPATH_TO_RELEVANT_URL_TOKENS = 'data/pretrain_data/relevant_url_token_counts.csv'
FPATH_to_HEAD_ROBOTS = "data/robots/temporal_robots_head.json"
FPATH_TO_RAND_ROBOTS = "data/robots/temporal_robots_rand_10k.json"
FPATH_TO_TOS_DATA = "data/GPT_analysis_results/tos_ai_scraping_policies.json"
FPATH_TO_TOS_LICENSE_DATA = "data/GPT_analysis_results/tos_license_policies.json"
FPATH_TO_TOS_COMPETE_DATA = "data/GPT_analysis_results/tos_competing_services_policies.json"
FPATH_TO_C4_TOKEN_ESTIMATES = "data/raw_annotations/c4_total_token_estimates.csv" 
FPATH_TO_DOLMA_TOKEN_ESTIMATES = "data/raw_annotations/dolma_total_token_estimates.csv"
FPATH_TO_RF_TOKEN_ESTIMATES = "data/raw_annotations/rf_total_token_estimates.csv" 
DIRPATHS_TO_ANNOTATED_TASKS = ["data/raw_annotations/task_1", "data/raw_annotations/task_2"]
START_DATES = "data/raw_annotations/domain_start_dates.json"

ALL_COMPANIES_TO_TRACK = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta", "Internet Archive", "Google Search", "False Anthropic"]
COMPANIES_TO_ANALYZE = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta"]
TEMPORAL_ANALYSIS_START_DATE = '2016-01-01'
TEMPORAL_ANALYSIS_END_DATE = '2024-04-30'

### Load all URL splits (top vs random) and maps to Token Counts

In [ ]:
url_token_lookup = robots_util.URLTokenLookup(FPATH_TO_RELEVANT_URL_TOKENS) # 'c4', 'rf', 'dolma'
c4_url_to_counts = url_token_lookup.get_url_to_token_map("c4")
rf_url_to_counts = url_token_lookup.get_url_to_token_map("rf")
dolma_url_to_counts = url_token_lookup.get_url_to_token_map("dolma")
top_c4_urls = url_token_lookup.top_k_urls("c4", 2000)
top_rf_urls = url_token_lookup.top_k_urls("rf", 2000)
top_dolma_urls = url_token_lookup.top_k_urls("dolma", 2000)
random_10k_urls = url_token_lookup.get_10k_random_sample()
all_urls = set(random_10k_urls + top_c4_urls + top_rf_urls + top_dolma_urls)

website_start_dates = robots_util.read_start_dates(START_DATES, all_urls)

### Define Agents and Agent Groups

In [ ]:
agent_groups_to_track = robots_util.get_bot_groups(ALL_COMPANIES_TO_TRACK)
agent_subset_to_track = robots_util.get_bot_groups(COMPANIES_TO_ANALYZE)
agents_to_track = robots_util.get_bots()

### Load Robots.txt info

In [ ]:
# URL -> Date -> Robots.txt raw text
head_robots = io.read_json(FPATH_to_HEAD_ROBOTS)
random_10k_robots = io.read_json(FPATH_TO_RAND_ROBOTS)
joined_robots = copy.deepcopy(head_robots)
joined_robots.update(random_10k_robots)
robots_util.print_out_robots_info(head_robots)
robots_util.print_out_robots_info(random_10k_robots)

# {URL --> Date --> Agent --> Status}
url_robots_summary, agent_counter_df = robots_util.compute_url_date_agent_status(
    data=joined_robots,
    # relevant_agents=agents_to_track)
    relevant_agents=[v for vs in agent_groups_to_track.values() for v in vs])

agent_counter_df.to_csv("all_agents_counter.csv", index=False)

In [ ]:
url_robots_summary_detailed = robots_util.compute_url_date_agent_status_detailed(
    data=joined_robots,
    relevant_agents=[v for vs in agent_groups_to_track.values() for v in vs]
)

### Load ToS info

In [ ]:
# URL --> Date --> ToS-suburl --> {"verdict": X, "evidence": Y}
tos_policies = {robots_util.normalize_url(url): info for url, info in io.read_json(FPATH_TO_TOS_DATA).items()}
tos_license_policies = {robots_util.normalize_url(url): info for url, info in io.read_json(FPATH_TO_TOS_LICENSE_DATA).items()}
tos_compete_policies = {robots_util.normalize_url(url): info for url, info in io.read_json(FPATH_TO_TOS_COMPETE_DATA).items()}
# tos_license_policies = robots_util.switch_dates_yearly_to_monthly(tos_license_policies)
print(f"Num ToS AI/Scraping URLs: {len(tos_policies)}")
print(f"Num ToS License URLs: {len(tos_license_policies)}")
print(f"Num ToS Compete URLs: {len(tos_compete_policies)}")

### Load Manual Pretraining Annotations

In [ ]:
url_to_info, unannotated_urls = analysis_util.extract_url_annotations(DIRPATHS_TO_ANNOTATED_TASKS)
url_results_df = analysis_util.process_url_annotations(url_to_info)
url_results_df = analysis_util.encode_size_columns(url_results_df, url_token_lookup)
manually_annotated_urls = url_results_df["URL"].tolist()
url_results_df = robots_util.encode_latest_tos_robots_into_df(
    url_results_df, tos_policies, tos_license_policies, tos_compete_policies, url_robots_summary,
    COMPANIES_TO_ANALYZE
)

service_to_urls = analysis_util.map_services_to_urls(url_results_df)

# Create Plots

### Preprocessing for Robots Head & Random URL splits

In [ ]:
### DECISION POINT: Use C4, Dolma, or RefinedWeb here?

CHOSEN_CORPUS = "c4" # 'c4', 'rf', 'dolma'
if CHOSEN_CORPUS == "c4":
    HEAD_URL_SET = top_c4_urls
    URL_TO_COUNTS = c4_url_to_counts
elif CHOSEN_CORPUS == "rf":
    HEAD_URL_SET = top_rf_urls
    URL_TO_COUNTS = rf_url_to_counts
elif CHOSEN_CORPUS == "dolma":
    HEAD_URL_SET = top_dolma_urls
    URL_TO_COUNTS = dolma_url_to_counts

In [ ]:
url_robots_summary_head = {url: url_robots_summary[url] for url in HEAD_URL_SET if url in url_robots_summary}
url_robots_summary_head_detailed = {url: url_robots_summary_detailed[url] for url in HEAD_URL_SET if url in url_robots_summary_detailed}
url_robots_summary_rand = {url: url_robots_summary[url] for url in random_10k_urls if url in url_robots_summary}
url_robots_summary_rand_detailed = {url: url_robots_summary_detailed[url] for url in random_10k_urls if url in url_robots_summary_detailed}

In [ ]:
DETAILED_ROBOTS_STRICTNESS_ORDER = [
    'no_robots', 'none', 'none_sitemap', 'none_crawl_delay', 'some_pattern_restrictions', 'some_disallow_important_dir', 'some_other', 'all'
]

# C4
urlsubset_to_robots_summary_c4, c4_url_subsets = robots_util.prepare_temporal_robots_for_corpus(
    url_robots_summary_detailed,
    top_c4_urls,
    random_10k_urls,
    service_to_urls,
    c4_url_to_counts,
    agent_groups_to_track,
    DETAILED_ROBOTS_STRICTNESS_ORDER,
    TEMPORAL_ANALYSIS_START_DATE,
    TEMPORAL_ANALYSIS_END_DATE,
    website_start_dates,   
)
# REFINEDWEB
urlsubset_to_robots_summary_rf, rf_url_subsets = robots_util.prepare_temporal_robots_for_corpus(
    url_robots_summary_detailed,
    top_rf_urls,
    random_10k_urls,
    service_to_urls,
    rf_url_to_counts,
    agent_groups_to_track,
    DETAILED_ROBOTS_STRICTNESS_ORDER,
    TEMPORAL_ANALYSIS_START_DATE,
    TEMPORAL_ANALYSIS_END_DATE,
    website_start_dates,   
)
# DOLMA
urlsubset_to_robots_summary_dolma, dolma_url_subsets = robots_util.prepare_temporal_robots_for_corpus(
    url_robots_summary_detailed,
    top_dolma_urls,
    random_10k_urls,
    service_to_urls,
    dolma_url_to_counts,
    agent_groups_to_track,
    DETAILED_ROBOTS_STRICTNESS_ORDER,
    TEMPORAL_ANALYSIS_START_DATE,
    TEMPORAL_ANALYSIS_END_DATE,
    website_start_dates,   
)

In [ ]:
# C4
urlsubset_to_tos_summary_c4, c4_tos_url_subsets = robots_util.prepare_temporal_tos_for_corpus(
    tos_policies,
    tos_license_policies,
    tos_compete_policies,
    top_c4_urls,
    random_10k_urls,
    service_to_urls,
    c4_url_to_counts,
    agent_groups_to_track,
    TEMPORAL_ANALYSIS_START_DATE,
    TEMPORAL_ANALYSIS_END_DATE,
    manually_annotated_urls,
    website_start_dates,
)
# REFINEDWEB
urlsubset_to_tos_summary_rf, rf_tos_url_subsets = robots_util.prepare_temporal_tos_for_corpus(
    tos_policies,
    tos_license_policies,
    tos_compete_policies,
    top_rf_urls,
    random_10k_urls,
    service_to_urls,
    rf_url_to_counts,
    agent_groups_to_track,
    TEMPORAL_ANALYSIS_START_DATE,
    TEMPORAL_ANALYSIS_END_DATE,
    manually_annotated_urls,
    website_start_dates,
)
# DOLMA
urlsubset_to_tos_summary_dolma, dolma_tos_url_subsets = robots_util.prepare_temporal_tos_for_corpus(
    tos_policies,
    tos_license_policies,
    tos_compete_policies,
    top_dolma_urls,
    random_10k_urls,
    service_to_urls,
    dolma_url_to_counts,
    agent_groups_to_track,
    TEMPORAL_ANALYSIS_START_DATE,
    TEMPORAL_ANALYSIS_END_DATE,
    manually_annotated_urls,
    website_start_dates,
)

In [ ]:
# HEAD URL SPLIT
# {Period --> Agent --> Status --> set(URLs)}
robots_filled_status_head_summary = robots_util.prepare_robots_temporal_summary(
    url_robots_summary=url_robots_summary_head,
    # group_to_agents={k: [k] for k in agents_to_track},
    group_to_agents=agent_groups_to_track,
    start_time=TEMPORAL_ANALYSIS_START_DATE,
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)
# RANDOM URL SPLIT
robots_filled_status_rand_summary = robots_util.prepare_robots_temporal_summary(
    url_robots_summary=url_robots_summary_rand,
    # group_to_agents={k: [k] for k in agents_to_track},
    group_to_agents=agent_groups_to_track,
    start_time=TEMPORAL_ANALYSIS_START_DATE,
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)

# DETAILED HEAD
robots_filled_status_head_summary_detailed = robots_util.prepare_robots_temporal_summary_detailed(
    url_robots_summary=url_robots_summary_head_detailed,
    group_to_agents=agent_groups_to_track,
    start_time=TEMPORAL_ANALYSIS_START_DATE,
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)

# DETAILED RAND
robots_filled_status_rand_summary_detailed = robots_util.prepare_robots_temporal_summary_detailed(
    url_robots_summary=url_robots_summary_rand_detailed,
    group_to_agents=agent_groups_to_track,
    start_time=TEMPORAL_ANALYSIS_START_DATE,
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)

In [ ]:
# Dataframe w/ [Period, Agent, Status, count(URLs or tokens)]
robots_temporal_head_summary = robots_util.robots_temporal_to_df(
    robots_filled_status_head_summary,
    strictness_order=['no_robots', 'none', 'some', 'all'],
    url_to_counts=URL_TO_COUNTS,
)
# Dataframe w/ [Period, Agent, Status, count(URLs), count(tokens)]
robots_temporal_rand_summary = robots_util.robots_temporal_to_df(
    robots_filled_status_rand_summary,
    strictness_order=['no_robots', 'none', 'some', 'all'],
    url_to_counts=URL_TO_COUNTS,
)

detailed_strictness_order =  [
    "no_robots",
    "none",
    "none_sitemap",
    "none_crawl_delay",
    "some_other",
    "some_disallow_important_dir",
    "some_pattern_restrictions",
    "all"
]
robots_temporal_head_summary_detailed = robots_util.robots_temporal_to_df(
    robots_filled_status_head_summary_detailed,
    strictness_order=detailed_strictness_order,
    url_to_counts=URL_TO_COUNTS,
)
robots_temporal_rand_summary_detailed = robots_util.robots_temporal_to_df(
    robots_filled_status_rand_summary_detailed,
    strictness_order=detailed_strictness_order,
    url_to_counts=URL_TO_COUNTS,
) 

### Preprocessing for ToS

In [ ]:
# URL --> time --> ToS verdict string.
url_to_time_to_tos_verdict = robots_util.get_tos_url_time_verdicts(
    tos_policies,
    tos_license_policies,
    tos_compete_policies,
    manually_annotated_urls,
    website_start_dates)
# Period --> Status --> set(URLs)
period_tos_verdict_urls = robots_util.prepare_tos_temporal_summary(
    url_to_time_to_tos_verdict,
    start_time=TEMPORAL_ANALYSIS_START_DATE,
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)
# Dataframe: [Period, Status, Count, Tokens]
tos_summary_df_head = robots_util.tos_temporal_to_df(
    period_tos_verdict_urls,
    url_set=HEAD_URL_SET,
    url_to_counts=URL_TO_COUNTS,
)
tos_summary_df_rand = robots_util.tos_temporal_to_df(
    period_tos_verdict_urls,
    url_set=random_10k_urls,
    url_to_counts=URL_TO_COUNTS,
)

## Plotting Constants

#### Event Markers

In [ ]:
# Test Events (Date, Text)
# EVENTS = [
#     ("2016-08-01", "Event A"),
#     ("2020-06-01", "Event B"),
#     ("2021-06-01", "Event C"),
#     ("2022-07-01", "Event D"),
#     ("2023-02-01", "Event E")
# ]

EVENTS = []

#### Plot Style

In [ ]:
PLOT_WIDTH = 1000
PLOT_HEIGHT = 160
LABEL_FONTSIZE = 14
TITLE_FONTSIZE = 16
LEGEND_COLS = 6


PLOT_KWARGS = {
    "width": PLOT_WIDTH,
    "height": PLOT_HEIGHT,
    "label_fontsize": LABEL_FONTSIZE,
    "title_fontsize": TITLE_FONTSIZE,
    "legend_cols": LEGEND_COLS,
    "vertical_line_dates": EVENTS
}

#### Forecasting Hyperparameters

In [ ]:
# ANALYSIS_TYPE = "autoregression"
# ANALYSIS_TYPE = "prophet"
# ANALYSIS_TYPE = "arima"
ANALYSIS_TYPE = "sarima"
# ANALYSIS_TYPE = None # No forecasting
LAGS = [1, 3, 6, 12, 24, 48]
ORDER = (2, 1, 2)
SEASONAL_ORDER = (1, 1, 1, 6)
N_PERIODS = 12 # Number of periods to forecast (months, by default)

#### Helper Function for Forecasting

In [ ]:
def forecast_plot(
    df: pd.DataFrame,
    agent: str,
    lags: typing.List[int],
    val_col: str = "tokens", # "count" / "tokens"
    n_periods: int = 6,
    ordered_statuses: typing.List[str] = None,
    status_colors: typing.Dict[str, str] = None,
    chosen_corpus: str = "c4",
    **kwargs
) -> alt.Chart:
    """Forecast and plot time series data (robots plots).

    Args:
        df (pd.DataFrame): Dataframe with columns like period, agent, status, count
        agent (str): Which agent to use
        lags (list[int]): List of lags to include in autoregression model
        val_col (str, optional): Column to forecast. Defaults to "tokens".
        n_periods (int, optional): Number of periods to forecast. Defaults to 6.
        seaonal_order (tuple[int], optional): Seasonal order for SARIMA. Defaults to None, which means no seasonal component.
        ordered_statuses (list[str], optional): Order of statuses in the legend.
        status_colors (dict[str, str], optional): Mapping of status to plot colors.

    Returns:
        alt.Chart: Altair chart
    """
    if ANALYSIS_TYPE == "autoregression":
        chart, _ = forecasting_util.forecast_and_plot(
            df,
            agent=agent,
            lags=lags,
            status_colors=status_colors,
            ordered_statuses=ordered_statuses,
            val_col=val_col,
            n_periods=n_periods,
            detailed=True,
            **kwargs
        )
    elif ANALYSIS_TYPE == "prophet":
        chart = forecasting_util.forecast_and_plot_prophet(
            df,
            agent=agent,
            lags=lags,
            status_colors=status_colors,
            ordered_statuses=ordered_statuses,
            n_periods=n_periods,
            **kwargs
        )
    elif ANALYSIS_TYPE == "arima":
        chart = forecasting_util.forecast_and_plot_arima(
            df,
            agent=agent,
            lags=lags,
            status_colors=status_colors,
            ordered_statuses=ordered_statuses,
            n_periods=n_periods,
            **kwargs
        )
    elif ANALYSIS_TYPE == "sarima":
        chart = forecasting_util.forecast_and_plot_sarima(
            df,
            agent=agent,
            order=ORDER,
            seasonal_order=SEASONAL_ORDER,
            status_colors=status_colors,
            ordered_statuses=ordered_statuses,
            n_periods=n_periods,
            chosen_corpus=chosen_corpus,
            **kwargs
        )
    else:
        chart = robots_util.plot_robots_time_map_altair_detailed(
            df,
            agent_type=agent,
            period_col="period",
            status_col="status",
            val_col="tokens", # "count" / "tokens"
            ordered_statuses=ordered_statuses,
            status_colors=status_colors,
            **kwargs
        )

    return chart

#### Other Parameters

In [ ]:
AGENT = "Combined Agent"

In [ ]:
def times_newroman():
    font = "Times New Roman"

    return {
          "config" : {
               "title": {"font": font},
               "axis": {
               "labelFont": font,
               "titleFont": font
          },
          "header": {
               "labelFont": font,
               "titleFont": font
          },
          "legend": {
               "labelFont": font,
               "titleFont": font
          },
          "text": {
               "font": font
          }
     }
}

alt.themes.register("times_newroman", times_newroman)
alt.themes.enable("times_newroman")

## Plot Temporal Robots Chart

In [ ]:
ORDERED_STATUSES = [
    "no_robots",
    "none",
    "none_sitemap",
    "none_crawl_delay",
    "some_other",
    "some_disallow_important_dir",
    "some_pattern_restrictions",
    "all"
]

STATUS_COLORS = {
    "all": "#a32a31",
    "some_pattern_restrictions": "#c76753",
    "some_disallow_important_dir": "#f7dcca",
    "some_other": "#f7f7f7",
    "none_crawl_delay": "#d5e4ef",
    "none_sitemap": "#9dc4db",
    "none": "#5891bf",
    "no_robots": "#d9d9d9"
}

LEGEND_MAPPING = {
    "all": "Full restrictions",
    "some_pattern_restrictions": "Pattern-based restrictions",
    "some_disallow_important_dir": "Disallow private directories",
    "some_other": "Other restrictions",
    "none_crawl_delay": "Crawl delay specified",
    "none_sitemap": "Sitemap provided",
    "none": "No restrictions or sitemap",
    "no_robots": "No Robots.txt"
}


# Copy and remap data for plotting
# df = robots_temporal_rand_summary_detailed.copy()
df = urlsubset_to_robots_summary_c4["head-all"].copy()
df["status"] = df["status"].map(LEGEND_MAPPING)
ordered_statuses = [LEGEND_MAPPING[s] for s in ORDERED_STATUSES]
status_colors = {LEGEND_MAPPING[k]: v for k, v in STATUS_COLORS.items()}

chart_robots = forecast_plot(
    df,
    agent=AGENT,
    lags=LAGS,
    val_col="tokens",
    n_periods=N_PERIODS,
    ordered_statuses=ordered_statuses,
    status_colors=status_colors,
    legend_title="Robots.txt Agent Restrictions",
    excel_file=EXCEL_FNAME_ROBOTS,
    chosen_corpus=CHOSEN_CORPUS,
    **PLOT_KWARGS
)


#chart_robots.save(
#    "%s_%s-%s.pdf" % (AGENT, CHOSEN_CORPUS, ANALYSIS_TYPE),
#    ppi=300
#)

chart_robots

## Plot Temporal ToS Area Chart + Forecast

In [ ]:
ORDERED_STATUSES = [
    "No Terms Pages", "Unrestricted Use", "Conditional Use", "No Re-Distribution", "Non-Compete", "NC Only", "No AI", "No Scraping", "No Scraping & AI"
]

STATUS_COLORS = {
    "No Terms Pages": "#d9d9d9",
    "Unrestricted Use": "#5891bf",
    "Conditional Use": "#d5e4ef",
    "No Re-Distribution": "#f7dcca",
    "Non-Compete": "#f7dcca",
    "NC Only": "#f7dcca",
    "No AI": "#f7dcca",
    "No Scraping": "#e8a888",
    "No Scraping & AI": "#a32a31"
}

# 'No Terms Pages' 'Unrestricted Use' 'No Scraping + NC Only'
#  'Conditional Use' 'NC Only' 'No Scraping' 'No Scraping & AI + NC Only'
#  'No Scraping & AI' 'No AI']


df = urlsubset_to_tos_summary_c4["head-all"].copy()


chart_tos = forecasting_util.plot_and_forecast_tos_sarima(
    df,
    period_col="period",
    status_col="status",
    val_col="tokens",  # "count" / "tokens"
    title="",
    ordered_statuses=ORDERED_STATUSES,
    status_colors=STATUS_COLORS,
    order=ORDER,
    seasonal_order=SEASONAL_ORDER,
    configure=False,
    legend_title="Web Domain Terms of Service Restrictions",
    excel_file=EXCEL_FNAME_TOS,
    chosen_corpus=CHOSEN_CORPUS,
    **PLOT_KWARGS
)

#chart_tos.save(
#    "TOS_%s.png" % (ANALYSIS_TYPE),
#    ppi=300
#)

chart_tos

## Final Predictions

In [ ]:
total_estimates = pd.read_csv(
    FPATH_TO_C4_TOKEN_ESTIMATES if CHOSEN_CORPUS == "c4" else FPATH_TO_RF_TOKEN_ESTIMATES if CHOSEN_CORPUS == "rf" else FPATH_TO_DOLMA_TOKEN_ESTIMATES,
    parse_dates=["period"],
)

forecast_df = forecasting_util.forecast_restrictive_tokens_sarima(
    total_estimates,
    excel_file=EXCEL_FNAME_FINAL,
    chosen_corpus=CHOSEN_CORPUS,
    seasonal_order=SEASONAL_ORDER,
    order=(2, 1, 1) if CHOSEN_CORPUS == "c4" else (1, 2, 1),
)

## Plot Restrictions by Company

In [ ]:
# df = robots_temporal_head_summary.copy()
df = urlsubset_to_robots_summary_c4["head-all"].copy()

forecast_startdate = df["period"].max().strftime("%Y-%m-%d")
if ANALYSIS_TYPE == "autoregression":
    df = forecasting_util.forecast_company_comparisons_autoregression(
        df,
        lags=LAGS,
        val_col="tokens",
        n_periods=N_PERIODS
    )
elif ANALYSIS_TYPE == "sarima":
    df = forecasting_util.forecast_company_comparisons_sarima(
        df,
        val_col="tokens",
        n_periods=N_PERIODS,
        order=ORDER,
        seasonal_order=SEASONAL_ORDER,
    )

AGENT_COLORMAP = {
    "Google": "#4D58D5",          # blue
    "OpenAI": "#519BEE",          # coral
    "Anthropic": "#68D4C4",       # green
    "Cohere": "#83F590",          # red
    "Common Crawl": "#DEDD56",    # muted purple
    "Meta": "#ADF86C",            # slate gray
    "Internet Archive": "#5D6D7E",# golden yellow
    "Google Search": "#D7EDEE",   # light gray
    "False Anthropic": "#D9D9D9"  # teal
}

chart_companies = visualization_util.plot_company_comparisons_altair(
    df,
    color_mapping=AGENT_COLORMAP,
    scale_y="log",
    configure=False,
    eventline_scaling=4, # How tall the event marker lines should be (scaling is relative to the span of data at the event time)
    forecast_startdate=forecast_startdate,
    skip_pct=True,
    legend_title="Restrictions by Organization Agent",
    **PLOT_KWARGS
)

#chart_companies.save(
#    "Company_Comparisons_%s.png" % (ANALYSIS_TYPE),
#    ppi=300
#)

chart_companies

## Combine Charts

In [ ]:
combined_chart = alt.vconcat(chart_robots, chart_tos, chart_companies).configure_axis(
    grid=False,
    labelFontSize=PLOT_KWARGS["label_fontsize"],
    titleFontSize=PLOT_KWARGS["title_fontsize"],
    labelAngle=0,
    tickCount=len(range(df["period"].dt.year.min(), df["period"].dt.year.max()))
).configure_legend(
    labelFontSize=PLOT_KWARGS["label_fontsize"],
    titleFontSize=PLOT_KWARGS["title_fontsize"],
    columns=PLOT_KWARGS["legend_cols"]
).configure_view(
    strokeWidth=0
).resolve_scale(
    color="independent",
    x="shared"
)

#combined_chart.save(
#    "Combined_%s.png" % (ANALYSIS_TYPE),
#    ppi=300
#)

combined_chart